In [ ]:
import sys
sys.path.append('..')
from configure.settings import DBSelector
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Bar, Line
from pyecharts.charts import Scatter
import pyecharts.options as opts
from pyecharts.charts import Line, Grid
from pyecharts.commons.utils import JsCode
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
)

area_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
)
db = DBSelector()

In [ ]:
engine = db.get_engine('db_stock')

In [ ]:
df = pd.read_sql('tb_bond_jisilu',con=engine)

In [ ]:
df.sample()

In [ ]:
# 溢价率 =（转债现价 - 转股价值）/ 转股价值
def stock_value(bond_price,bondPremRatio):
    return bond_price/(1+bondPremRatio)
    

In [ ]:
stock_value(173.698,-17.39/100)

In [ ]:
file = '/home/xda/hub/convertible_bond/data/new_bond.xlsx'

In [ ]:
data = pd.read_excel(file)

In [ ]:
data.sample()

In [ ]:
data = data.drop(0)

In [ ]:
data.columns

In [ ]:
data.sample()

In [ ]:
data = data.drop('Unnamed: 0',axis=1)

In [ ]:
data.sample()

In [ ]:
data['innerValue'] = data['closePriceBond']/(1+data['bondPremRatio']/100)

In [ ]:
data.sample()

# 曲线部分

In [ ]:
X = data['innerValue']
Y=data['closePriceBond']

In [ ]:
scatter = (
    Scatter(init_opts=opts.InitOpts(width="1024px", height="768px",bg_color=JsCode(background_color_js)))
    .add_xaxis(xaxis_data=X)
    .add_yaxis(
        
        series_name="上市价格",
        y_axis=Y,
        symbol_size=10,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_series_opts()
    .set_global_opts(
        title_opts=opts.TitleOpts(title='转债上市价格预测'),
        xaxis_opts=opts.AxisOpts(
            type_="value",
            name='转债转股价值',
            splitline_opts=opts.SplitLineOpts(is_show=True),
            min_=60,
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name='转债价格收盘价',
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
            min_=60,
            
        ),
        tooltip_opts=opts.TooltipOpts(is_show=False),
    )
)
scatter.render('only.html')

In [ ]:

plt.figure(figsize=(12,8))

plt.scatter(X,Y)
plt.xlabel('innerValue')
plt.ylabel('closePriceBond')
plt.show()

In [ ]:
z1 = np.polyfit(X,Y,3) #用3次多项式拟合  可以改为5 次多项式。。。。 返回三次多项式系数
p1= np.poly1d(z1)
print(p1) #在屏幕上打印拟合多项式
yvals = p1(X) #也可以使用yvals=np.polyval(z1,x)

In [ ]:
length = len(X)
print(length)
X2 = [X[i] for i in range(0,length,1)]
# X2=
yvals = p1(X2) #也可以使用yvals=np.polyval(z1,x)
plt.figure(figsize=(12,8))

plot1 = plt.plot(X,Y,'.',label='original values')

plot2 = plt.plot(X2,yvals,'*',label='polyfit values')

plt.xlabel('xaxis')

plt.ylabel('yaxis')

plt.legend(loc=4)  #指定legend的位置,读者可以自己help它的用法

plt.title('polyfitting')
plt.savefig('ploy.png')

In [ ]:
data.info()

In [ ]:
x=range(10)
y=list(map(lambda x:x**2,x))

In [ ]:
plt.scatter(x,y)

In [ ]:
z2 = np.polyfit(x,y,2)
p2 = np.poly1d(z2)
print(p2)


In [ ]:
# 使用指数拟合
from scipy.optimize import curve_fit
def func(x,a,b):
      return a*np.exp(b/x)


popt,pcov=curve_fit(func,X,Y)

a=popt[0] #popt里面是拟合系数，读者可以自己help其用法

b=popt[1]

zs_yvals=func(X,a,b)

In [ ]:
func(93.78,a,b)

In [ ]:
func(90.60,a,b)

In [ ]:
a

In [ ]:
b

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(X,Y,'*')
plt.plot(X,yvals,'*')

In [ ]:
predict_value =func(93.78,a,b)

In [ ]:
predict_value

# pyecharts

In [ ]:
scatter = (
    Scatter(init_opts=opts.InitOpts(width="1024px", height="768px",bg_color=JsCode(background_color_js)))
    .add_xaxis(xaxis_data=X)
    .add_yaxis(
        series_name="1111",
        y_axis=Y,
        symbol_size=3,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_series_opts()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            type_="value", 
            splitline_opts=opts.SplitLineOpts(is_show=True),
            min_=60,
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
            min_=60,
            
        ),
        tooltip_opts=opts.TooltipOpts(is_show=False),
    )
)

# scatter.render_notebook()


In [ ]:
X0=X[::10]
Y0=Y[::10]
zs_yvals0 = zs_yvals[::10]

In [ ]:
sort_XY = list(sorted(zip(X,zs_yvals),key=lambda x:x[0]))

In [ ]:
sort_XY[0]

In [ ]:
X0=[item[0] for item in sort_XY]
Y0=[item[1] for item in sort_XY]

In [ ]:
scatter = (
    Scatter(init_opts=opts.InitOpts(width="1024px", height="768px",bg_color=JsCode(background_color_js)))
    .add_xaxis(xaxis_data=X)
    .add_yaxis(
        series_name="上市价格",
        y_axis=Y,
        symbol_size=5,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_series_opts()
    .set_global_opts(
        title_opts=opts.TitleOpts(title='转债上市价格预测'),
        xaxis_opts=opts.AxisOpts(
            name='转股价值',
            type_="value", 
            splitline_opts=opts.SplitLineOpts(is_show=True),
            min_=60,
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name='转债上市首日收盘价',
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
            min_=60,
            
        ),
        tooltip_opts=opts.TooltipOpts(is_show=False),
    )
)

line = (
    Line()
    .add_xaxis(xaxis_data=X0)
    .add_yaxis(
        series_name="拟合曲线",
        # yaxis_index=1,
        symbol_size=5,
        y_axis=Y0,
        label_opts=opts.LabelOpts(is_show=False),
        linestyle_opts=opts.LineStyleOpts(width=1,color='yellow'),
        
    ).set_colors("yellow") 
)
line2 = (
    Line()
    .add_xaxis(xaxis_data=[60,180])
    .add_yaxis(
        series_name="溢价率为0",
        # yaxis_index=1,
        symbol_size=5,
        y_axis=[60,180],
        label_opts=opts.LabelOpts(is_show=False),
        linestyle_opts=opts.LineStyleOpts(width=2,color='green'),
        
    ).set_colors("green") 
)
scatter.overlap(line)
scatter.overlap(line2)
# scatter.render_notebook()
scatter.render('预测.html')

In [ ]:
X.max()

In [ ]:
Y.max()

In [ ]:
# plt.xlabel('innerValue')
# plt.ylabel('closePriceBond')
data['closePriceBond'].argmax()

In [ ]:
data.iloc[400]

In [ ]:
data['closePriceBond'].argmax()

In [ ]:
data.iloc[438]

In [ ]:
data['bondPremRatio'].argmax()

In [ ]:
data.iloc[1]

In [ ]:
len(data)

In [ ]:
def abs_error():
    count = len(Y)
    sum=0
    min_,min_index,max_,max_index=999999,0,0,0
    index=0
    for real,predict in zip(Y,zs_yvals):
        v=abs(real-predict)
        if v>max_:
            max_=v
            max_index=index
        if v<min_:
            min_=v
            min_index=index
        index+=1
        sum+=v
    return min_,min_index,max_,max_index,sum/count

In [ ]:
abs_error()

In [ ]:
data.iloc[400]

In [ ]:
Y[0]

In [ ]:
scatter = (
    Scatter(init_opts=opts.InitOpts(width="800px", height="600px"))
    .add_xaxis(xaxis_data=X)
    .add_yaxis(
        series_name="转债价格",
        y_axis=Y,
        symbol_size=3,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_series_opts()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            type_="value", splitline_opts=opts.SplitLineOpts(is_show=True),
            min_=60,
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
            min_=60,
            
        )
    )
)

line = (
    Line()
    .add_xaxis(xaxis_data=X0)
    .add_yaxis(
        series_name="可转债价格预测",
        # yaxis_index=1,
        symbol_size=5,
        y_axis=Y0,
        label_opts=opts.LabelOpts(is_show=False),
        linestyle_opts=opts.LineStyleOpts(width=1,color='yellow'),
        
    ).set_colors("yellow") 
)
scatter.overlap(line)
scatter.render_notebook()

In [ ]:
import pyecharts.options as opts
from pyecharts.charts import Line, Grid
from pyecharts.commons.utils import JsCode

"""
参考地址: https://gallery.echartsjs.com/editor.html?c=xEyDk1hwBx
"""

x_data = ["14", "15", "16", "17", "18", "19", "20", "21", "22", "23"]
y_data = [393, 438, 485, 631, 689, 824, 987, 1000, 1100, 1200]

background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
)
area_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
)

c = (
    Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="注册总量",
        y_axis=y_data,
        is_smooth=True,
        is_symbol_show=True,
        symbol="circle",
        symbol_size=6,
        linestyle_opts=opts.LineStyleOpts(color="#fff"),
        label_opts=opts.LabelOpts(is_show=True, position="top", color="white"),
        itemstyle_opts=opts.ItemStyleOpts(
            color="red", border_color="#fff", border_width=3
        ),
        tooltip_opts=opts.TooltipOpts(is_show=False),
        areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="我是标题",
            pos_bottom="5%",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(color="#fff", font_size=16),
        ),
        xaxis_opts=opts.AxisOpts(
            name='我是x轴呢',
            type_="category",
            boundary_gap=False,
            axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63"),
            axisline_opts=opts.AxisLineOpts(is_show=False),
            axistick_opts=opts.AxisTickOpts(
                is_show=True,
                length=25,
                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
            ),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
            ),
        ),
        yaxis_opts=opts.AxisOpts(
            name='我是y轴,右边的',
            type_="value",
            position="right",
            axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63"),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
            ),
            axistick_opts=opts.AxisTickOpts(
                is_show=True,
                length=15,
                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
            ),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
            ),
        ),
        legend_opts=opts.LegendOpts(is_show=False),
    )
)

(
    Grid()
    .add(
        c,
        grid_opts=opts.GridOpts(
            pos_top="20%",
            pos_left="10%",
            pos_right="10%",
            pos_bottom="15%",
            is_contain_label=True,
        ),
    )
    .render_notebook()
)